In [ ]:
%sql

-- ------------------------------------------------------------
-- 1. Filter for NO measurements at site GB6
-- ------------------------------------------------------------
SELECT 
    site,
    species,
    reading_datetime,
    value,
    units
FROM  `catalog-dev-uks-corecdp-001`.`schema-dev-uks-corecdp-gold-001`.london_environment_data
WHERE species = 'NO'
  AND site = 'GB6'
ORDER BY reading_datetime
LIMIT 5;


-- ------------------------------------------------------------
-- 3. Compute hourly average NO concentration
-- ------------------------------------------------------------
SELECT
    site,
    DATE_TRUNC('hour', reading_datetime) AS hour,
    AVG(value) AS mean_value
FROM  `catalog-dev-uks-corecdp-001`.`schema-dev-uks-corecdp-gold-001`.london_environment_data
WHERE species = 'NO'
GROUP BY
    site,
    DATE_TRUNC('hour', reading_datetime)
ORDER BY
    site,
    hour
LIMIT 5;


In [ ]:
%r
# ------------------------------------------------------------
# Example R Analysis Workflow for Databricks (sparklyr + dplyr)
# ------------------------------------------------------------

# Load required libraries
library(sparklyr)
library(dplyr)
library(lubridate)
library(ggplot2)

# ------------------------------------------------------------
# 1. Connect to Spark in Databricks
# ------------------------------------------------------------
sc <- spark_connect(method = "databricks")

# ------------------------------------------------------------
# 2. Load the data table from the Databricks catalog
# ------------------------------------------------------------
df <- tbl(sc,  "`catalog-dev-uks-corecdp-001`.`schema-dev-uks-corecdp-gold-001`.london_environment_data")

# ------------------------------------------------------------
# 3. Preview a few rows of the dataset
# ------------------------------------------------------------
df %>%
  head(5) %>%
  collect()

# ------------------------------------------------------------
# 4. Filter for NO measurements from site GB6
# ------------------------------------------------------------
no_readings <- df %>%
  filter(species == "NO", site == "GB6") %>%
  select(site, species, reading_datetime, value, units)

# Show filtered sample
no_readings %>%
  head(10) %>%
  collect()

# ------------------------------------------------------------
# 5. Calculate hourly average NO concentration
# ------------------------------------------------------------
hourly_avg <- df %>%
  filter(species == "NO") %>%
  mutate(hour = date_trunc("hour", reading_datetime)) %>%   # Truncate to the hour
  group_by(site, hour) %>%
  summarise(mean_value = mean(value, na.rm = TRUE)) %>%
  arrange(hour)

# Show sample of hourly averages
hourly_avg %>%
  head(10) %>%
  collect()

# ------------------------------------------------------------
# 6. Convert Spark results to a local R dataframe for plotting
# ------------------------------------------------------------
hourly_avg_local <- hourly_avg %>% collect()

# ------------------------------------------------------------
# 7. Plot the hourly trend using ggplot2
# ------------------------------------------------------------
ggplot(hourly_avg_local, aes(x = hour, y = mean_value, color = site)) +
  geom_line() +
  labs(
    title = "Hourly Average NO Levels",
    x = "Hour",
    y = "Mean NO (µg/m³)"
  ) +
  theme_minimal()
